# Todobackend Walkthrough

### Visual Studio Code

Set `python.terminal.activateEnvironment` setting to false

### Create Django Project

### Git Setup

Create git repository:

Create `.gitignore` File:

Add the following to `.gitignore`:

Comment .vscode settings in  `.gitignore`:

### Install Django REST

Install Django components.  Virtual environment will be automatically created by `pipenv`

Activate pipenv shell:

### Create Django Application

Create application called todo:

Update installed apps in `src/todobackend/settings.py`:

In [ ]:
INSTALLED_APPS = (
     ...
     'rest_framework',
     'corsheaders',
     'todo'
)

# cars middleware must be before CommonMiddleware
MIDDLEWARE = (
    ...
    'corsheaders.middleware.CorsMiddleware',
    'django.middleware.common.CommonMiddleware',
     ...
)

# CORS Settings

CORS_ORIGIN_ALLOW_ALL = True

### Develop Application

Create TodoItem model in `src/todo/models.py`:

In [ ]:
from django.db import models

# Create your models here.
class TodoItem(models.Model):
    """
    TodoItem Model
    """
    title = models.CharField(max_length=256, null=True, blank=True)
    completed = models.BooleanField(blank=True, default=False)
    url = models.CharField(max_length=256, null=True, blank=True)
    order = models.IntegerField(null=True, blank=True)

Create and run migrations:

Create Serializer in `src/todo/serializers.py`:

In [ ]:
from rest_framework import serializers

from todo.models import TodoItem

class TodoItemSerializer(serializers.HyperlinkedModelSerializer):
    url = serializers.ReadOnlyField()
    class Meta:
        model = TodoItem
        fields = ('url', 'title', 'completed', 'order')

Create Views in `src/todo/views.py`:

In [ ]:
from django.shortcuts import render
from rest_framework import status
from rest_framework import viewsets
from rest_framework.reverse import reverse
from rest_framework.response import Response

from todo.models import TodoItem
from todo.serializers import TodoItemSerializer

# Create your views here.
class TodoItemViewSet(viewsets.ModelViewSet):
    queryset = TodoItem.objects.all()
    serializer_class = TodoItemSerializer

    def perform_create(self, serializer):
        """
        Creates a todo item
        """
        # Save instance to get primary key and then update URL
        instance = serializer.save()
        instance.url = reverse('todoitem-detail', args=[instance.pk], request=self.request)
        instance.save()

    def delete(self, request):
        """
        Deletes all todo items
        """
        TodoItem.objects.all().delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

Create main routes in `src/todobackend/urls.py`:

In [ ]:
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('todo.urls')),
]

Create todo routes `src/todo/urls.py`:

In [ ]:
from django.conf.urls import url, include
from rest_framework.routers import DefaultRouter

from todo import views

# Create a router and register our viewsets with it.
router = DefaultRouter(trailing_slash=False)
router.register(r'todos', views.TodoItemViewSet)

# The API URLs are now determined automatically by the router.
urlpatterns = [
    url('', include(router.urls)),
]

### Create Tests

Create `src/todo/tests.py`:

In [ ]:
from django.test import TestCase
from django.urls import reverse
from rest_framework import status
from rest_framework.test import APITestCase
from todo.models import TodoItem


def createItem(client):
    url = reverse('todoitem-list')
    data = {'title': 'Walk the dog'}
    return client.post(url, data, format='json')


class TestCreateTodoItem(APITestCase):
    """
    Ensure we can create a new todo item
    """
    def setUp(self):
        self.response = createItem(self.client)
    
    def test_received_201_created_status_code(self):
        self.assertEqual(self.response.status_code, status.HTTP_201_CREATED)

    def test_received_location_header_hyperlink(self):
        self.assertRegexpMatches(self.response['Location'], '^http://.+/todos/[\d]+$')

    def test_item_was_created(self):
        self.assertEqual(TodoItem.objects.count(), 1)

    def test_item_has_correct_title(self):
        self.assertEqual(TodoItem.objects.get().title, 'Walk the dog')


class TestUpdateTodoItem(APITestCase):
    """
    Ensure we can update an existing todo item using PUT
    """
    def setUp(self):
        response = createItem(self.client)
        self.assertEqual(TodoItem.objects.get().completed, False)
        url = response['Location']
        data = {'title': 'Walk the dog', 'completed': True}
        self.response = self.client.put(url, data, format='json')

    def test_received_200_created_status_code(self):
        self.assertEqual(self.response.status_code, status.HTTP_200_OK)

    def test_item_was_updated(self):
        self.assertEqual(TodoItem.objects.get().completed, True)


class TestPatchTodoItem(APITestCase):
    """
    Ensure we can update an existing todo item using PATCH
    """
    def setUp(self):
        response = createItem(self.client)
        self.assertEqual(TodoItem.objects.get().completed, False)
        url = response['Location']
        data = {'title': 'Walk the dog', 'completed': True}
        self.response = self.client.patch(url, data, format='json')
    
    def test_received_200_ok_status_code(self):
        self.assertEqual(self.response.status_code, status.HTTP_200_OK)
    
    def test_item_was_updated(self):
        self.assertEqual(TodoItem.objects.get().completed, True)


class TestDeleteTodoItem(APITestCase):
    """
    Ensure we can delete a todo item
    """
    def setUp(self):
        response = createItem(self.client)
        self.assertEqual(TodoItem.objects.count(), 1)
        url = response['Location']
        self.response = self.client.delete(url)

    def test_received_204_no_content_status_code(self):
        self.assertEqual(self.response.status_code, status.HTTP_204_NO_CONTENT)

    def test_the_item_was_deleted(self):
        self.assertEqual(TodoItem.objects.count(), 0)


class TestDeleteAllItems(APITestCase):
    """
    Ensure we can delete all todo items
    """
    def setUp(self):
        createItem(self.client)
        createItem(self.client)
        self.assertEqual(TodoItem.objects.count(), 2)
        self.response = self.client.delete(reverse('todoitem-list'))

    def test_received_204_no_content_status_code(self):
        self.assertEqual(self.response.status_code, status.HTTP_204_NO_CONTENT)

    def test_all_items_were_deleted(self):
        self.assertEqual(TodoItem.objects.count(), 0)

Run tests:

### Refactor Settings

- Create `todobackend/settings` folder
- Add `todobackend/settings/__init__.py` blank file
- Add `todobackend/settings/base.py`
- Copy `todobackend/settings.py` to `todobackend/settings/base.py`

Update `manage.py`:

In [ ]:
#!/usr/bin/env python
"""Django's command-line utility for administrative tasks."""
import os
import sys


def main():
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'todobackend.settings.base')
    try:
        from django.core.management import execute_from_command_line
    except ImportError as exc:
        raise ImportError(
            "Couldn't import Django. Are you sure it's installed and "
            "available on your PYTHONPATH environment variable? Did you "
            "forget to activate a virtual environment?"
        ) from exc
    execute_from_command_line(sys.argv)


if __name__ == '__main__':
    main()


Update `todobackend/wsgi.py`:

In [ ]:
"""
WSGI config for todobackend project.

It exposes the WSGI callable as a module-level variable named ``application``.

For more information on this file, see
https://docs.djangoproject.com/en/3.0/howto/deployment/wsgi/
"""

import os

from django.core.wsgi import get_wsgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'todobackend.settings.base')

application = get_wsgi_application()


Update `todobackend/asgi.py` (only applies for Django 3.x):

In [ ]:
"""
ASGI config for todobackend project.

It exposes the ASGI callable as a module-level variable named ``application``.

For more information on this file, see
https://docs.djangoproject.com/en/3.0/howto/deployment/asgi/
"""

import os

from django.core.asgi import get_asgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'todobackend.settings.base')

application = get_asgi_application()


### Install MariaDB

In [ ]:
# Fix for mysqlserver.stop not working
# See https://stackoverflow.com/questions/59936589/how-can-i-fix-brew-installed-mariadb-that-hangs-on-mysql-server-stop-and-doesn/59938033#59938033
cp /usr/local/bin/mysql.server /usr/local/bin/mysql.server.backup
sed -i "" "s/user='mysql'/user=\`whoami\`/g" /usr/local/bin/mysql.server

Create database and user:

In [ ]:
% mysql -u root -p 
MariaDB [(none)]> CREATE DATABASE todobackend;
MariaDB [(none)]> GRANT ALL PRIVILEGES ON *.* TO 'todo'@'localhost' identified by 'password';
MariaDB [(none)]> quit

### Install Additional Dependencies

In [ ]:
# Gotcha - remove any misnamed packages from Pipfile, they are not automatically removed
pipenv install mysql-connector-python
pipenv install django-nose --dev
pipenv install pinocchio --dev
pipenv install coverage --dev

### Add Test Settings

Add to `todobackend/settings/test.py`:

In [ ]:
from .base import *
import os

# Installed Apps
INSTALLED_APPS += ('django_nose', )
TEST_RUNNER = 'django_nose.NoseTestSuiteRunner'
TEST_OUTPUT_DIR = os.environ.get('TEST_OUTPUT_DIR', '.')
NOSE_ARGS = [
    '--verbosity=2',                                        # verbose output
    '--nologcapture',                                       # don't output log capture
    '--with-coverage',                                      # activate coverage report
    '--cover-package=todo',                                 # coverage reports will apply to these packages
    '--with-spec',                                          # spec style tests
    '--spec-color',                                         # make them pretty
    '--with-xunit',                                         # enable xunit plugin
    f'--xunit-file={TEST_OUTPUT_DIR}/nosetests.xml',        # xunit test output
    '--cover-xml',                                          # produce XML coverage info
    f'--cover-xml-file={TEST_OUTPUT_DIR}/coverage.xml',     # XML coverage output
]

# Database
# https://docs.djangoproject.com/en/1.8/ref/settings/#databases
DATABASES = {
    'default': {
        # Supports AWS X-Ray
        'ENGINE': 'mysql.connector.django',
        'NAME': os.environ.get('MYSQL_DATABASE','todobackend'),
        'USER': os.environ.get('MYSQL_USER','todo'),
        'PASSWORD': os.environ.get('MYSQL_PASSWORD','password'),
        'HOST': os.environ.get('MYSQL_HOST','localhost'),
        'PORT': os.environ.get('MYSQL_PORT','3306'),
        # See https://code.djangoproject.com/ticket/30469#comment:5
        'OPTIONS': {
            'use_pure': True
        }
    }
}

Run tests using new settings:

### Acceptance Testing

Install `pytest` and `requests`:

In [ ]:
pipenv install pytest requests --dev

### Additional Stuff

Add the following to `.env` file to import code in ipython and ensure VS Code can resolve code: